In [ ]:
import random
from itertools import product, accumulate
import numpy as np
import matplotlib.pyplot as plt
from random import choices
from icecream import ic
import math


UNIVERSE_SIZE = 100000
NUM_SETS = 10000
DENSITY = 0.1   #each set has 10% to cover that point     #0.3      


np.random.seed(23)
random.seed(23) 
rng = np.random.Generator(np.random.PCG64([UNIVERSE_SIZE, NUM_SETS, int(10_000 * DENSITY)]))



# DON'T EDIT THESE LINES!

SETS = np.random.random((NUM_SETS, UNIVERSE_SIZE)) < DENSITY
for s in range(UNIVERSE_SIZE):
    if not np.any(SETS[:, s]):
        SETS[np.random.randint(NUM_SETS), s] = True
COSTS = np.power(SETS.sum(axis=1), 1.1)




def valid(solution):
    """Checks wether solution is valid (ie. covers all universe)"""
    return np.all(np.logical_or.reduce(SETS[solution]))         # We are taking or not all sets
                                                                # and we are considering the union of all this sets
    # Here the fitness is if all items are taken or not
    
    # Then I check if I have taken all objects


def cost(solution):
    """Returns the cost of a solution (to be minimized)"""
    return COSTS[solution].sum()    # If I check the number of covered items => integer from 0 going up and saturating to the number of point
                                    # I only go up, don't have basin of attraction that are terrible, but still we hava problems with local optima

def fitness(solution: np.ndarray):
    return (valid(solution),-cost(solution))   # I am trying to minimize the cost => so if I want to MAXIMIZE the fitness I use -

In [77]:

# Calculation of the frequencies of the nodes and evaluating the weights. 
freq = SETS.sum(axis=0) # how many cluster contain the node
weights = 1/freq  # less frequent = more important

# Calculation of the scores of each sets
initial_scores = SETS @ weights

# Calculation of ‘artificial’ probabilities using softmax
exp_scores = np.exp(initial_scores)
sum_exp_scores = np.sum(exp_scores)
PROBS = exp_scores / sum_exp_scores
solution = np.full(NUM_SETS, False)




In [78]:

def new_tweak(solution, num) -> np.ndarray:       
    
    new_sol = solution.copy() 

    if  not valid(solution): # Adding sets if you need to include nodes 
        probs = PROBS.copy()
        probs = probs*(~solution)  # Probability with respect to only the unselected sets
        probs = probs/np.sum(probs)  
        indexes  = choices(np.arange(NUM_SETS), weights=probs, k=num)
        new_sol[indexes] = ~new_sol[indexes]
        return new_sol
    
    else:   # Removing sets if you don't need to include nodes 
        inv_probs = PROBS.copy()
        inv_probs = inv_probs*solution  # Probability with respect to only the selected sets
        inv_probs = inv_probs/np.sum(inv_probs)
        indexes = choices(np.arange(NUM_SETS), weights=inv_probs, k=num)
        new_sol[indexes] = ~new_sol[indexes] 
        return new_sol
    



In [79]:
def tweak(solution: np.ndarray) -> np.ndarray: 
    new_sol = solution.copy()
    i = rng.integers(0,NUM_SETS)
    new_sol[i] = not new_sol[i]
    return new_sol

In [ ]:
best_solution = np.full(NUM_SETS, False)
best_fitness = -NUM_SETS*UNIVERSE_SIZE

solution_fitness = 0
history = []
temperature = 1000
rate_temperature = 0.98

num_tweak = int(min(0.005*NUM_SETS, 1))

for out_it in range(30):
    print(out_it)
    solution = np.full(NUM_SETS, False)
    initialization_probs = PROBS.copy()
    
    # Let’s take the first valid solution assuming we are making selections without replacement from the set.
    while not valid(solution):
        chosen_index = choices(np.arange(NUM_SETS), weights=initialization_probs, k=1)
        solution[chosen_index] = True
        if np.all(solution):
            initialization_probs[chosen_index] = 0
            initialization_probs = initialization_probs/np.sum(initialization_probs)
            initialization_probs[-1] += 1 - np.sum(initialization_probs)
    
    solution_fitness = fitness(solution)
    history.append(solution_fitness[1])

    new_solution = solution.copy()
    for in_it in range(1000):
        # PART INFLUENCED BY PROBABILITY (simulated annealing is implicit in the for loop)
        for i in range(5): 
            new_solution = new_tweak(new_solution, num_tweak)
            f = fitness(new_solution)
            
            if f > fitness(solution) and valid(new_solution):
                history.append(f[1])   
                #print(f"1 - outer iteration: {out_it}")
                solution = new_solution.copy()
                solution_fitness = f
 
        
        # HILL CLIMBING PART (explicit simulated annealing)
        new_solution = tweak(solution)
        f = fitness(new_solution)
 
        if f > fitness(solution) and valid(new_solution):
            history.append(f[1]) 
            #print(f"2 - outer iteration: {out_it}")
            solution = new_solution.copy()
            solution_fitness = f
 
        else:
            delta = f[1] - fitness(solution)[1]
            acceptance_probability = math.exp(delta / temperature)
            # simulated annealing
            if np.random.rand() < acceptance_probability and valid(new_solution):
                solution = new_solution.copy()
                solution_fitness = f
    
    if best_fitness < solution_fitness[1]:
        best_solution = solution.copy()
        best_fitness = solution_fitness[1]
    
    
    temperature *= rate_temperature




#ic(best_solution)
ic(best_fitness)

plt.figure(figsize = (14,8))
plt.plot(
    range(len(history)),
    list(accumulate(history, max)),
    color = "red"
)

_ = plt.scatter(range(len(history)), history, marker = ".")


RESULTS

100  10  0.2   <br>
-286.437      6.3 s   out_it=30   in_it=1000 

1000 100 0.2 <br>
  -10587.550      10.1 s   out_it=30   in_it=1000  

10000 100 0.2  <br>
 -117930.432     32.6 s   out_it=30   in_it=1000

100000 10000 0.1 <br>
 -1851363.949  9 min 42.2 s  out_it=30 in_it=1000

100000 10000 0.2 <br>
-2034141.269  10 min 49.3 s  out_it=30 in_it=1000

100000 10000 0.3 <br>
-2012269.240 11 min 7.5 s  out_it=30 in_it=1000





